In [326]:
import pandas as pd
import tabula

In [327]:
dfs = tabula.read_pdf("syumatsu2023092900.pdf", lattice=True, pages = 'all')

PDFのページを一枚のdataframeへ結合する。  
セル結合により、NaNデータの発生、カラム名の損失などが起きている。  

空dataframeを作成する。  
その後、NaNデータやint型に変換不可能な列を持つ行は削除する。

In [328]:
concated_df = pd.DataFrame()
for df in dfs:
    concated_df = pd.concat([concated_df, df.dropna()])

次に、カラム名を変更する。

In [329]:
cols = ["銘柄名", "新証券コード", "合計売残", "合計売残前週比", "合計買残", "合計買残前週比", "一般売残", "一般売残前週比", "制度売残", "制度売残前週比", "一般買残", "一般買残前週比", "制度買残", "制度買残前週比"]
concated_df.columns = cols

In [330]:
concated_df.head()


,銘柄名,新証券コード,合計売残,合計売残前週比,合計買残,合計買残前週比,一般売残,一般売残前週比,制度売残,制度売残前週比,一般買残,一般買残前週比,制度買残,制度買残前週比
6,B極洋 普通株式13010,JP3257200000,"4,100","▲ 1,000","26,600",▲ 400,200,0,"3,900","▲ 1,000","12,900","▲ 2,300","13,700","1,900"
7,Bニッスイ 普通株式13320,JP3718800000,"329,900","▲ 46,600","920,300","2,900","41,900","▲ 5,100","288,000","▲ 41,500","389,200","▲ 18,800","531,100","21,700"
8,Bマルハニチロ 普通株式13330,JP3876600002,"58,100","▲ 4,800","603,200","17,300","37,700","▲ 2,400","20,400","▲ 2,400","429,400","2,800","173,800","14,500"
9,Bサカタのタネ 普通株式13770,JP3315000004,"27,700",▲ 200,"30,100","▲ 18,400","20,600",▲ 400,"7,100",200,"11,200","▲ 3,000","18,900","▲ 15,400"
10,Bホクト 普通株式13790,JP3843250006,"28,700","1,500","36,100","▲ 4,400","24,100",0,"4,600","1,500","6,200","▲ 1,200","29,900","▲ 3,200"


次は、計算しやすいように数字を数値に成形する  

In [331]:
concated_df = concated_df.map(lambda x: x.strip() if isinstance(x, str) else x)
concated_df = concated_df.map(lambda x: x.replace('▲ ', '-'))
concated_df = concated_df.map(lambda x: x.replace(',', ''))

In [332]:
concated_df.head()

,銘柄名,新証券コード,合計売残,合計売残前週比,合計買残,合計買残前週比,一般売残,一般売残前週比,制度売残,制度売残前週比,一般買残,一般買残前週比,制度買残,制度買残前週比
6,B極洋 普通株式13010,JP3257200000,4100,-1000,26600,-400,200,0,3900,-1000,12900,-2300,13700,1900
7,Bニッスイ 普通株式13320,JP3718800000,329900,-46600,920300,2900,41900,-5100,288000,-41500,389200,-18800,531100,21700
8,Bマルハニチロ 普通株式13330,JP3876600002,58100,-4800,603200,17300,37700,-2400,20400,-2400,429400,2800,173800,14500
9,Bサカタのタネ 普通株式13770,JP3315000004,27700,-200,30100,-18400,20600,-400,7100,200,11200,-3000,18900,-15400
10,Bホクト 普通株式13790,JP3843250006,28700,1500,36100,-4400,24100,0,4600,1500,6200,-1200,29900,-3200


PDFの都合上、カラムが破壊されて文字化けしているケースもある。

In [333]:
filtered_df = concated_df
as_int = filtered_df.columns[2:]
for column in as_int:
    d = filtered_df[column].apply(lambda x: isinstance(x, int) or x.isdigit())
    filtered_df = filtered_df[d]

普通株式以外は必要ないため、「普通株式」を含まない行を削除する。  

In [334]:
filtered_df = filtered_df[filtered_df.iloc[:, 0].str.contains('普通株式', na=False)]


銘柄列に、接頭辞や普通株式や証券番号が入り込んでいるので削除

In [335]:
filtered_df["銘柄名"] = filtered_df["銘柄名"].str.replace("B\s*", "", regex=True)
filtered_df["銘柄名"] = filtered_df["銘柄名"].str.replace("\s.*$", "", regex=True)

In [336]:
filtered_df.head()

,銘柄名,新証券コード,合計売残,合計売残前週比,合計買残,合計買残前週比,一般売残,一般売残前週比,制度売残,制度売残前週比,一般買残,一般買残前週比,制度買残,制度買残前週比
34,飛島建設,JP3629800008,14600,2700,385400,5600,100,0,14500,2700,307100,500,78300,5100
43,大豊建設,JP3498600000,9400,0,10400,1700,6700,0,2700,0,5900,700,4500,1000
54,日本道路,JP3740200005,26000,20600,39600,34000,0,0,26000,20600,11900,10400,27700,23600
5,日本国土開発,JP3707200006,19500,1900,332300,14100,600,600,18900,1300,194200,11100,138100,3000
13,ライト工業,JP3965800000,28900,100,68900,2500,23500,0,5400,100,18400,1000,50500,1500


3列目以降の型をstring型からint型に変換する。

In [337]:
filtered_df.iloc[:,2:] = filtered_df.iloc[:, 2:].astype(int)

In [338]:
filtered_df.to_csv("output.csv", index=False)